In [8]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from sklearn.neighbors import KDTree


In [2]:
quasar_all = np.load('../catalogue/quasar_lss_all.npy')
quasar_high = np.load('../catalogue/quasar_lss_high_L.npy')
quasar_low = np.load('../catalogue/quasar_lss_low_L.npy')

In [4]:
quasar_zbin = [np.load(f'../catalogue/quasar_lss_z{i}.npy') for i in range(1, 4)]

In [10]:
quasar_SDSS = fits.getdata('/uufs/chpc.utah.edu/common/home/astro/zheng/hd/data/SDSS16Q/DR16Q_v4.fits')
tree = KDTree(np.c_[quasar_SDSS['RA'], quasar_SDSS['DEC']], metric='euclidean')

In [6]:
quasar_ls = [
    quasar_all, 
    quasar_low,
    quasar_high,
    *quasar_zbin
]

In [11]:
q = quasar_all
que = tree.query(np.c_[q['ra'], q['dec']])
valid = que[0] < 5/3600
index = que[1]
M_I = quasar_SDSS['M_I'][index].astype(float)
M_I[~valid] = np.nan
M_I = M_I.flatten()

In [12]:
M_I

array([-26.08581321, -26.65745419, -26.68262305, ..., -25.65601557,
       -25.41499148, -24.35537206])

In [21]:
mean_MI(M_I[valid.flatten()])

-25.038733033455614

In [13]:
def mean_MI(MIs):
    L = 10**((MIs + 25)/(-2.5))
    return -2.5*np.log10(L.mean()) - 25


In [22]:
for q in quasar_ls:
    que = tree.query(np.c_[q['ra'], q['dec']])
    valid = (que[0] < 5/3600).flatten()
    index = que[1].flatten()
    M_I = quasar_SDSS['M_I'][index].astype(float)
    M_I[~valid] = np.nan
    print(len(q), q['z'].min(), q['z'].max(), q['z'].mean(), mean_MI(M_I[valid]))

343708 0.8000152398143878 2.199999421096105 1.512227653125804 -25.038733033455614
171733 0.8000152398143878 2.1999820553107012 1.51212736600451 -24.035606146427018
171765 0.8000399639105563 2.199999421096105 1.5125837882148954 -25.550973409639226
114569 0.8000152398143878 1.3123750214556185 1.084231590681535 -24.30328838268877
114569 1.3123764939283518 1.7168144990224052 1.513662399529402 -24.974316616987522
114569 1.7168150781021316 2.199999320222159 1.9387829660437461 -25.514037424309265
